In [1]:
import pickle
from pandas import DataFrame
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Layout
import pathlib

plt.rcParams["figure.figsize"] = (14, 7) # (w, h)
pair_address = "0xEBFb684dD2b01E698ca6c14F10e4f289934a54D6"
cache_df_path = f"data/{pair_address}_by_block_df.pkl"
sync_swap_df = f"data/{pair_address}_df.pkl"

state_df = pickle.load(open(sync_swap_df, "rb"))
state_df.head()

def index_by_block_and_cache(state_df: DataFrame) -> DataFrame:
    block_df = (state_df[['block', 'amount0In', 'amount1In', 'amount0Out', 'amount1Out', 'reserve0', 'reserve1']]).groupby(
        state_df.block, as_index=False).agg({
            'block': 'first', 'amount0In': sum, 'amount1In': sum, 'amount0Out': sum, 'amount1Out': sum, 'reserve0': 'first', 'reserve1': 'first'
        }).copy(deep=True)
    block_df.block = block_df.block.astype(int)

    block_df = block_df.set_index('block').reindex(np.arange(block_df.block.min(), block_df.block.max() + 1)).reset_index()
    block_df.reserve0 = block_df.reserve0.fillna(method='bfill')
    block_df.reserve1 = block_df.reserve1.fillna(method='bfill')
    block_df = block_df.fillna(value=0)
    block_df.to_pickle(cache_df_path)
    return block_df

# block_df is 172M due to intoducing a row for each block, build it and cache it to run cell faster on future runs.
block_df = pickle.load(open(cache_df_path, "rb")) \
    if pathlib.Path(cache_df_path).exists() \
    else index_by_block_and_cache(pickle.load(open(sync_swap_df, "rb")))

#rates_df = DataFrame({
#    "block": block_df["block"],
#    "y_per_x": block_df["reserve1"] / block_df["reserve0"],
#    "x_per_y": block_df["reserve0"] / block_df["reserve1"]
#})


In [10]:
#block_df.head()
#len(state_df)
filtered_df = state_df[(state_df['reserve0'] == 0) & (state_df['reserve1'] == 0)]
filtered_df.head()

,sender,to,amount0In,amount1In,amount0Out,amount1Out,block,reserve0,reserve1
0,0x00000047bB99ea4D791bb749D970DE71EE0b1A34,0xd3d2E2692501A5c9Ca623199D38826e513033a17,0,89730000,13202197110903059021,0,19017444,0,0


# Reserve based price vs Volume weighted average price

Price is usually measured by the state of the reserves at the end of a block. Another way to meausre price is considering the rate each swap executes at. This can be chaotic with multiple swaps in either direction in a block, but we might group these rates within a block using a volume weighted average. Lets compare how these values would differ from a reserved based price

In [ ]:
prices_df = DataFrame(block_df["block"])

# Add reserve based price
prices_df["reserve0/reserve1"] = block_df["reserve0"] / block_df["reserve1"]

# Add Volume based price, use last price if no volume exists
prices_df["volume0/volume1"] = (block_df["amount0In"] + block_df["amount0Out"]).divide(
    (block_df["amount1In"] + block_df["amount1Out"]).replace({ 0 : np.nan })).fillna(method='ffill')

# seperated volume values
prices_df["volume_sell_0_0per1"] = (block_df["amount0In"]).divide(
    (block_df["amount1Out"]).replace({ 0 : np.nan })).fillna(method='ffill')
prices_df["volume_sell_1_0per1"] = (block_df["amount0Out"]).divide(
    (block_df["amount1In"]).replace({ 0 : np.nan })).fillna(method='ffill')
prices_df["volume_sell_0_1per0"] = 1 / prices_df["volume_sell_0_0per1"]
prices_df["volume_sell_1_1per0"] = 1 / prices_df["volume_sell_1_0per1"]

prices_df = prices_df.set_index('block')

In [ ]:
prices_df.head()

In [ ]:
prices_df[["volume0/volume1", "reserve0/reserve1"]] \
    [(16300150 <= prices_df.index) & (prices_df.index <= 16300450)].plot(linewidth=2)

In [ ]:
price_compare = prices_df[["reserve0/reserve1", "volume_sell_0_0per1", "volume_sell_1_0per1", "volume0/volume1"]] \
    [(16300300 <= prices_df.index) & (prices_df.index <= 16300450)]
plt.plot(price_compare[["reserve0/reserve1"]], label = "reserve0/reserve1", linewidth=5)
plt.plot(price_compare[["volume_sell_0_0per1", "volume_sell_1_0per1"]], label = ["volume_sell_0_0per1", "volume_sell_1_0per1"], linewidth=4)
plt.plot(price_compare[["volume0/volume1"]], label = "volume0/volume1", linewidth=2)
plt.xlabel("block")
plt.ylabel('price')
plt.legend()
plt.show()

# 30 Minute Oracle Comparison on ETH-POOL Uniswap V2 Pair

Here we look at the ETH-POOL pair, a low liquidity trading pair, to understand how variout price calculations for our oracle would work. 

## Construct Oracles

Our dataframe starts with swaps and reserves prior to swaps. Using this data we crunch some numbers to build build various oraclse. Since we do not have time which our scanner does not grab, we use blocks thus BWAP rather than TWAP. If every block was 12 seconds exactly, this would be the same, but since that is not the case, there are some minor erorrs. 

In [ ]:
class OracleBuilder:
    tick_log = math.log(1.0001)

    def __init__(self, block_df: DataFrame):
        self.block_df = block_df
        self.df = DataFrame(block_df.loc[:, ("block")])
    
    @staticmethod
    def new_builder(block_df: DataFrame) -> 'OracleBuilder':
        return OracleBuilder(block_df)
    
    def add_reserve_price(self) -> 'OracleBuilder':
        self.df["current_price"] = (self.block_df['reserve0'] / self.block_df['reserve1'])
        return self
    
    def add_shifted_reserve_price(self, minutes) -> 'OracleBuilder':
        half_blocks = int(self.minutes_to_blocks(minutes) / 2)
        current_price = self.df["current_price"] if "current_price" in self.df.columns else self.block_df['reserve0'] / self.block_df['reserve1']
        self.df[f"shifted_price_{half_blocks}"] = current_price.shift(periods=half_blocks)
        return self
    
    def add_twap(self, minutes) -> 'OracleBuilder':
        blocks = self.minutes_to_blocks(minutes)
        cumsum_x_per_y = self.block_df.loc[:, ('reserve0')].divide(self.block_df['reserve1']).cumsum()
        cumsum_y_per_x = self.block_df.loc[:, ('reserve1')].divide(self.block_df['reserve0']).cumsum()
        self.df[f"twap_{minutes}_x_per_y"] = (cumsum_x_per_y - cumsum_x_per_y.shift(blocks)).divide(blocks)
        return self
    
    def add_geom_twap(self, minutes) -> 'OracleBuilder':
        blocks = self.minutes_to_blocks(minutes)
        tick_sum = (np.log((self.block_df['reserve0'] / self.block_df['reserve1']).astype(float)) / self.tick_log).cumsum()
        self.df[F"geom_twap_{minutes}_x_per_y"] = np.power(
            1.0001, (tick_sum - tick_sum.shift(blocks)) / blocks
        )
        return self
    
    def add_vwap(self, minutes) -> 'OracleBuilder':
        blocks = self.minutes_to_blocks(minutes)
        cumsum_x_in = self.block_df["amount0In"].cumsum()
        cumsum_y_out = self.block_df["amount1Out"].cumsum()
        cumsum_y_in = self.block_df["amount1In"].cumsum()
        cumsum_x_out = self.block_df["amount0Out"].cumsum()
        self.df[f"vwap_{minutes}_x_per_y"] = (
            cumsum_x_in + cumsum_x_out - (cumsum_x_in + cumsum_x_out).shift(blocks)).divide(
            (cumsum_y_in + cumsum_y_out - (cumsum_y_in + cumsum_y_out).shift(blocks)).replace({ 0 : np.nan }))
        return self
    
    def add_proposed(self, minutes) -> 'OracleBuilder':
        blocks = self.minutes_to_blocks(minutes)
        geom_0In_1Out = (np.log((self.block_df["amount0In"] / self.block_df["amount1Out"].replace({0: np.nan})).fillna(method='ffill')) / self.tick_log).cumsum()
        geom_1In_0Out = (np.log((self.block_df["amount0Out"] / self.block_df["amount1In"].replace({0: np.nan})).fillna(method='ffill')) / self.tick_log).cumsum()
        self.df[f'proposed_0In_1Out_oracle_{minutes}'] = np.power(1.0001, (geom_0In_1Out - geom_0In_1Out.shift(blocks)) / blocks)
        self.df[f'proposed_1In_0Out_oracle_{minutes}'] = np.power(1.0001, (geom_1In_0Out - geom_1In_0Out.shift(blocks)) / blocks)
        return self

    @staticmethod
    def minutes_to_blocks(minutes):
        return minutes * 5 # minutes * 60 second / 12 second per block

    def build(self) -> DataFrame:
        return self.df


oracle_df = OracleBuilder.new_builder(block_df) \
    .add_reserve_price() \
    .add_shifted_reserve_price(30) \
    .add_twap(30) \
    .add_geom_twap(30) \
    .add_vwap(30) \
    .add_proposed(30) \
    .build()

In [ ]:
oracle_df[["block", "current_price"]].plot(linewidth=.75, x="block", ylabel="POOL/ETH", title="POOL per ETH Reserves Based Price Uniswap V2 Pair")

# Proposed oracle calculationsin comparison to twap and geometric twap

Our proposed oracle calculation tracks what assets are tracking rather than what the state of the reserves in the pair are. Below we show each oracle calculation along with the price based on the reserves shifted forward to overlay with what the oracles which lag. The twaps break from the reserves briefly until the opposite direction trades occur at which point the twaps start decreaeasing towards the new reserve prices. The rate at which the swaps execute at are higher than the values the twaps track. The proposed oracle more closely follows the actual price, not the reserve based price. 

In [ ]:
oracle_df.drop('current_price', axis=1)[(16298700 <= oracle_df.block) & (
    # oracle_df.index <= 16300600
    oracle_df.block <= 16298900

)].plot(x="block")


In [ ]:
oracle_df.drop('current_price', axis=1)[(16300300 <= oracle_df.block) & (
    oracle_df.block <= 16300600
)].plot(x="block")

# Largest price moves in one block

Below we look at the largest moves in one block. Our first list has 17/20 in the first 24 hours of the pair getting created. 

In [ ]:
big_moves_df = DataFrame({
    "block": block_df["block"],
    "amount0In_per_reserve0": block_df["amount0In"] / block_df["reserve0"],
    "amount1In_per_reserve1": block_df["amount1In"] / block_df["reserve1"],
    "amount0Net_per_reserve0": (block_df["amount0In"] - block_df["amount0Out"]) / block_df["reserve0"],
    "amount1Net_per_reserve1": (block_df["amount1In"] - block_df["amount1Out"]) / block_df["reserve1"],
})

big_moves_df["max_amountIn_per_reserve"] = big_moves_df[[
    "amount0In_per_reserve0", "amount1In_per_reserve1"
]].max(axis=1)
big_moves_df["max_amountNet_per_reserve"] = big_moves_df[[
    "amount0Net_per_reserve0", "amount1Net_per_reserve1"
]].max(axis=1)


big_moves_df.sort_values("max_amountNet_per_reserve", ascending=False).reset_index(drop=True)
big_moves_df.head()

Lets drop the first 24 hours to look at the largest moves after the market has found some equilibrium after initialization

In [ ]:
sorted_net_moves_df = big_moves_df.iloc[7200:].sort_values("max_amountNet_per_reserve", ascending=False).reset_index(drop=True)
sorted_net_moves_df.head(10)

## Examining some of the largest moves 

Below is a chart of the largest moves. 

In [ ]:
sorted_net_moves_df[(sorted_net_moves_df.index  <= 25)][["max_amountIn_per_reserve", "max_amountNet_per_reserve"]].plot()

#### Looking at how the market and oracles behave after the largest shift in price

In [ ]:
b = sorted_net_moves_df["block"][0]
block_df[(b -5 <= block_df["block"]) & (block_df["block"] <= b + 20)]

In [ ]:
oracle_df[(b - 5 <= oracle_df.block)&(oracle_df.block <= b + 300)] \
    .drop(["vwap_30_x_per_y"], axis=1) \
    .plot(x="block", figsize=(12,6))

#### Interesting series of trades to come back to and look at

price fluctuates betwee 0.01 ETH / POOL to 0.026 ETH/Poool 

In [ ]:
r = (11882900 <= oracle_df.block) & (oracle_df.block <=11883550)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot((oracle_df.drop(
    [
        'block',
        'current_price',
        # "shifted_price", 
        "vwap_30_x_per_y"], axis=1))[r])
b = 2900
